# BioExplorer - Import SONATA circuit into PostgreSQL database
![](../../bioexplorer_neurons_banner.png)

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm
import numpy as np
import libsonata
import morphio

### Create the DB schema
In the PostgreSQL database, create the schema using the provided [SQL statements](../../../../../storage/database/02_bioexplorer_neurons_schema.sql)

In [ ]:
data_folder = os.getenv('O1_DATA_ANATOMY')
population_name = 'S1nonbarrel_neurons'

### Open rat non-barrel somatosensory cortex SONATA Circuit

In [ ]:
circuit_config = os.path.join(data_folder, 'circuit_config.json')
circuit = libsonata.CircuitConfig.from_file(circuit_config)

### Import data into the DB

In [ ]:
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = population_name

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

#### Read SONATA input

In [ ]:
nodes = circuit.node_population(population_name)
ids = np.arange(0, nodes.size)

etypes = nodes.get_attribute('etype', list(ids))
mtypes = nodes.get_attribute('mtype', list(ids))
regions = nodes.get_attribute('region', list(ids))
layers = nodes.get_attribute('layer', list(ids))
morphologies = nodes.get_attribute('morphology', list(ids))
morphology_classes = nodes.get_attribute('morph_class', list(ids))
xs = nodes.get_attribute('x', list(ids))
ys = nodes.get_attribute('y', list(ids))
zs = nodes.get_attribute('z', list(ids))
oxs = nodes.get_attribute('orientation_x', list(ids))
oys = nodes.get_attribute('orientation_y', list(ids))
ozs = nodes.get_attribute('orientation_z', list(ids))
ows = nodes.get_attribute('orientation_w', list(ids))
synapse_classes = nodes.get_attribute('synapse_class', list(ids))

#### Import electrical types

In [ ]:
etypes_guids = dict()
with Session(engine) as session:
    session.execute('DELETE FROM %s.electrical_type' % db_schema)
    session.commit()

duplicates = list()
etypes_set = set(etypes)
count = 0
with Session(engine) as session:
    for etype in tqdm(etypes_set):
        try:
            session.execute(
                'INSERT INTO %s.electrical_type VALUES (:guid, :code)' % db_schema,
                {
                    'guid': count,
                    'code': etype
                }
            )
            etypes_guids[etype] = count
            count += 1
            session.commit()
        except Exception as e:
            duplicates.append(etype)

#### Import morphological types

In [ ]:
mtypes_guids = dict()
with Session(engine) as session:
    session.execute('DELETE FROM %s.morphological_type' % db_schema)
    session.commit()

duplicates = list()
mtypes_set = set(mtypes)
count = 0
with Session(engine) as session:
    for mtype in tqdm(mtypes_set):
        try:
            session.execute(
                'INSERT INTO %s.morphological_type VALUES (:guid, :code)' % db_schema,
                {
                    'guid': count,
                    'code': mtype
                }
            )
            mtypes_guids[mtype] = count
            count += 1
            session.commit()
        except Exception as e:
            duplicates.append(mtype)

#### Import regions

In [ ]:
regions_guids = dict()
with Session(engine) as session:
    session.execute('DELETE FROM %s.region' % db_schema)
    session.commit()

duplicates = list()
regions_set = set(regions)
count = 0
with Session(engine) as session:
    for region in tqdm(regions_set):
        try:
            session.execute(
                'INSERT INTO %s.region VALUES (:guid, :code)' % db_schema,
                {
                    'guid': int(count),
                    'code': region
                }
            )
            regions_guids[region] = count
            count += 1
            session.commit()
        except Exception as e:
            duplicates.append(region)

#### Import layers

In [ ]:
layers_guids = dict()
with Session(engine) as session:
    session.execute('DELETE FROM %s.layer' % db_schema)
    session.commit()

duplicates = list()
layers_set = set(layers)
count = 0
with Session(engine) as session:
    for layer in tqdm(layers_set):
        try:
            session.execute(
                'INSERT INTO %s.layer VALUES (:guid, :code)' % db_schema,
                {
                    'guid': int(count),
                    'code': int(layer)
                }
            )
            layers_guids[layer] = count
            count += 1
            session.commit()
        except Exception as e:
            duplicates.append(layer)

#### Import morphology classes

In [ ]:
morphology_classes_guids = dict()
with Session(engine) as session:
    session.execute('DELETE FROM %s.morphology_class' % db_schema)
    session.commit()

duplicates = list()
morphology_classes_set = set(morphology_classes)
count = 0
with Session(engine) as session:
    for morphology_class in tqdm(morphology_classes_set):
        try:
            session.execute(
                'INSERT INTO %s.morphology_class VALUES (:guid, :code)' % db_schema,
                {
                    'guid': count,
                    'code': etype
                }
            )
            morphology_classes_guids[morphology_class] = count
            count += 1
            session.commit()
        except Exception as e:
            duplicates.append(morphology_class)

#### Import synapse classes

In [ ]:
synapse_classes_guids = dict()
with Session(engine) as session:
    session.execute('DELETE FROM %s.synapse_class' % db_schema)
    session.commit()

duplicates = list()
synapse_classes_set = set(synapse_classes)
count = 0
with Session(engine) as session:
    for synapse_class in tqdm(synapse_classes_set):
        try:
            session.execute(
                'INSERT INTO %s.synapse_class VALUES (:guid, :code)' % db_schema,
                {
                    'guid': count,
                    'code': synapse_class
                }
            )
            synapse_classes_guids[synapse_class] = count
            count += 1
            session.commit()
        except Exception as e:
            duplicates.append(synapse_class)

#### Import morphologies

In [ ]:
morphologies_guids = dict()
with Session(engine) as session:
    session.execute('DELETE FROM %s.morphology' % db_schema)
    session.commit()

duplicates = list()
morphologies_set = set(morphologies)
count = 0
with Session(engine) as session:
    for morphology in tqdm(morphologies_set):
        try:
            session.execute(
                'INSERT INTO %s.morphology VALUES (:guid, :basename)' % db_schema,
                {
                    'guid': count,
                    'basename': morphology
                }
            )
            morphologies_guids[morphology] = count
            count += 1
            session.commit()
        except Exception as e:
            print(e)
            duplicates.append(morphology)
            break

In [ ]:
props = circuit.node_population_properties(population_name)
morphology_folder = props.morphologies_dir

In [ ]:
section_types = dict()
section_types[morphio.SectionType.soma] = 0
section_types[morphio.SectionType.axon] = 1
section_types[morphio.SectionType.basal_dendrite] = 2
section_types[morphio.SectionType.apical_dendrite] = 3

section_type_descriptions = dict()
section_type_descriptions[0] = 'Soma'
section_type_descriptions[1] = 'Axon'
section_type_descriptions[2] = 'Basal dendrite'
section_type_descriptions[3] = 'Apical dendrite'

In [ ]:
with Session(engine) as session:
    session.execute('DELETE FROM %s.section_type' % db_schema)
    session.commit()

duplicates = list()
with Session(engine) as session:
    for section_type in tqdm(section_types):
        try:
            guid = section_types[section_type]
            session.execute(
                'INSERT INTO %s.section_type VALUES (:guid, :description)' % db_schema,
                {
                    'guid': int(guid),
                    'description': section_type_descriptions[guid]
                }
            )
            session.commit()
        except Exception as e:
            print(e)
            duplicates.append(section_type)

In [ ]:
with Session(engine) as session:
    session.execute('DELETE FROM %s.section' % db_schema)
    session.commit()

with Session(engine) as session:
    for morphology_guid in tqdm(range(len(morphologies_guids))):
        morphology=morphologies[morphology_guid]
        morphology_file = os.path.join(morphology_folder, morphology + '.asc')
        m = morphio.Morphology(morphology_file)

        for section in m.sections:
            section_type_guid = section_types[section.type]
            parent = -1
            if not section.is_root:
                parent =  section.parent.id

            points = list()
            for i in range(len(section.points)):
                point = section.points[i]
                points.append(point[0])
                points.append(point[1])
                points.append(point[2])
                points.append(section.diameters[i])
            buffer = np.array(points, dtype=np.float32).tobytes()

            point = section.points[0]

            session.execute(
                'INSERT INTO %s.section VALUES (:morphology_guid, :section_guid, :section_parent_guid, :section_type_guid, :points, :x, :y, :z)' % db_schema,
                {
                    'morphology_guid': morphology_guid,
                    'section_guid': section.id,
                    'section_parent_guid': parent,
                    'section_type_guid': section_type_guid,
                    'points': buffer,
                    'x': float(point[0]),
                    'y': float(point[1]),
                    'z': float(point[2])
                }
            )
        if morphology_guid%10 == 0:
            session.commit()
    session.commit()

#### Import nodes

In [ ]:
print('Deleting existing nodes')
with Session(engine) as session:
    session.execute('DELETE FROM %s.node' % db_schema)
    session.commit()

errors = list()
with Session(engine) as session:
    for i in tqdm(range(nodes.size)):
        try:
            session.execute(
                'INSERT INTO %s.node VALUES ( \
                    :guid, :x, :y, :z, :rotation_x, :rotation_y, :rotation_z, :rotation_w, \
                    :morphology, :morphology_class, \
                    :etype, :mtype, :region, :layer, :synapse_class)' % db_schema,
                {
                    'guid': int(i),
                    'x': xs[i],
                    'y': ys[i],
                    'z': zs[i],
                    'rotation_x': oxs[i],
                    'rotation_y': oys[i],
                    'rotation_z': ozs[i],
                    'rotation_w': ows[i],
                    'morphology': int(morphologies_guids[morphologies[i]]),
                    'morphology_class': int(morphology_classes_guids[morphology_classes[i]]),
                    'etype': int(etypes_guids[etypes[i]]),
                    'mtype': int(mtypes_guids[mtypes[i]]),
                    'region': int(regions_guids[regions[i]]),
                    'layer': int(layers_guids[layers[i]]),
                    'synapse_class': int(synapse_classes_guids[synapse_classes[i]])
                }
            )
            if i%1000==0:
                session.commit()
        except Exception as e:
            print(e)
            errors.append(e)
            break
        session.commit()